In [1]:
!pip install opencv-python

In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow import keras
from tensorflow.keras import layers
import os
import glob
import cv2

# Splitting dataset (splitting paths leading to images)

## creating dataset with paths to spectograms

In [3]:
# specify path to spectogram images
PATH = r"C:\Users\Ilan\Documents\ITC\11.Final_Project\2. Dataset 2\Data\images_original"

In [4]:
genres = os.listdir(PATH)

In [5]:
# getting path of all file names in directories
files = {}
for genre in genres:
    files[genre] = glob.glob(fr"{PATH}\{genre}\*png")

In [6]:
# creating dataframe wit paths
df = pd.DataFrame(columns = ['path', 'rgb_data', 'label'])
for genre in genres:
    df_aux = pd.DataFrame(files[genre], columns = ['path'])
    df_aux['label'] = genre
    df = df.append(df_aux, ignore_index=True, sort=False)

# verifying number of rows
df['label'].value_counts()

rock         100
metal        100
country      100
disco        100
blues        100
hiphop       100
reggae       100
classical    100
pop          100
jazz          99
Name: label, dtype: int64

In [7]:
#getting rgb data for all images
for index in df.index:
    img = cv2.imread(df['path'].loc[index])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #df['rgb_data'].loc[index] = np.array(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), dtype=np.uint8)
    df['rgb_data'].loc[index] = np.asarray(img).astype('float32')

In [8]:
df.head()

,path,rgb_data,label
0,C:\Users\Ilan\Documents\ITC\11.Final_Project\2...,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",blues
1,C:\Users\Ilan\Documents\ITC\11.Final_Project\2...,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",blues
2,C:\Users\Ilan\Documents\ITC\11.Final_Project\2...,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",blues
3,C:\Users\Ilan\Documents\ITC\11.Final_Project\2...,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",blues
4,C:\Users\Ilan\Documents\ITC\11.Final_Project\2...,"[[[255.0, 255.0, 255.0], [255.0, 255.0, 255.0]...",blues


In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
# encoding label
le = LabelEncoder()
le.fit(genres)
df['label_encoded'] = le.transform(df['label'])

In [11]:
df['label_encoded']

0      0
1      0
2      0
3      0
4      0
      ..
994    9
995    9
996    9
997    9
998    9
Name: label_encoded, Length: 999, dtype: int32

## splitting dataset

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
#defining X and y
X = [data for data in df['rgb_data']]
X = np.array(X)
y = df['label_encoded']

#splitting data and stratifying by y
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Modelling

In [26]:
shape_of_input = df['rgb_data'][0].shape
shape_of_input

(288, 432, 3)

In [27]:
model = keras.Sequential([
    layers.Rescaling(1 / 255, input_shape=shape_of_input),
    
    layers.Conv2D(8, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=3),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(16, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=3),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=3),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(128, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
    layers.BatchNormalization(axis=-1),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    
    layers.Dropout(rate=0.3),
    
    layers.Dense(len(genres), activation='softmax')
])

In [29]:
print(f'Model input shape: {model.input_shape}, Model output shape: {model.output_shape}')

Model input shape: (None, 288, 432, 3), Model output shape: (None, 10)


In [30]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 288, 432, 3)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 286, 430, 8)       224       
_________________________________________________________________
batch_normalization_5 (Batch (None, 286, 430, 8)       32        
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 143, 215, 8)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 141, 213, 16)      1168      
_________________________________________________________________
batch_normalization_6 (Batch (None, 141, 213, 16)      64        
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 70, 106, 16)      

In [38]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = 'accuracy')
model.fit(X_train, y_train, validation_split=0.1, batch_size=16, epochs=5)

Epoch 1/5


ValueError: in user code:

    C:\Users\Ilan\anaconda3\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\Ilan\anaconda3\lib\site-packages\keras\engine\training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Ilan\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Ilan\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Ilan\anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Ilan\anaconda3\lib\site-packages\keras\engine\training.py:835 run_step  **
        outputs = model.train_step(data)
    C:\Users\Ilan\an